In [ ]:
## Normalized feature
features=pd.concat([BIall_v2,clinical],axis=0,sort=False).reset_index().iloc[:,1:].fillna(0)

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
features_norm = min_max_scaler.fit_transform(features)

fst_BI1=pd.DataFrame(features_norm).iloc[:-(123),:]
fst_cli1=pd.DataFrame(features_norm).iloc[-(123):,:].reset_index().iloc[:,1:]

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

# Load data and split in train and test sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(fst_BI2, Type_v2,test_size=0.2,random_state=42,stratify=Type_v2)
#X_train2, X_test2, y_train2, y_test2 = train_test_split(fst_BI2, Type,test_size=0.2,random_state=42,stratify=Type)

## Linear model

In [ ]:
from sklearn.model_selection import cross_val_score
lda_model = LinearDiscriminantAnalysis()
lda_preds = lda_model.fit(X_train, y_train).predict(X_test)

lda_acc = balanced_accuracy_score(y_test, lda_preds)
lda_preds_train=lda_model.fit(X_train, y_train).predict(X_train)
scores = cross_val_score(lda_model, X_train, y_train, cv=5)
lda_acc_train = balanced_accuracy_score(y_train, lda_preds_train)
print('LDA Test Accuracy: {}'.format(lda_acc))
print('LDA Training Accuracy: {}'.format(lda_acc_train))


from scipy import stats
rho_cli_lda, pval_cli_lda = stats.spearmanr(cli_tf,PSR.iloc[:,1])
print(rho_cli_lda, pval_cli_lda)

In [ ]:
qda_model = QuadraticDiscriminantAnalysis()
qda_preds = qda_model.fit(X_train, y_train).predict(X_test)
qda_acc = balanced_accuracy_score(y_test,qda_preds)
qda_preds_train=qda_model.fit(X_train, y_train).predict(X_train)
qda_acc_train = balanced_accuracy_score(y_train, qda_preds_train)
print('QDA Accuracy: {}'.format(qda_acc))
print('QDA Training Accuracy: {}'.format(qda_acc_train))
clinical_qda=qda_model.fit(X_train, y_train).predict(fst_cli)

In [ ]:
rda_model = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
rda_preds = rda_model.fit(X_train, y_train).predict(X_test)
rda_acc = balanced_accuracy_score(y_test, rda_preds)
rda_preds_train=rda_model.fit(X_train, y_train).predict(X_train)
rda_acc_train = balanced_accuracy_score(y_train, rda_preds_train)
print('RDA Accuracy: {}'.format(rda_acc))
print('RDA Training Accuracy: {}'.format(rda_acc_train))
clinical_rda=rda_model.fit(X_train, y_train).predict(fst_cli)

In [ ]:
logreg_model = LogisticRegression()
logreg_preds = logreg_model.fit(X_train, y_train).predict(X_test)
logreg_acc = balanced_accuracy_score(y_test, logreg_preds)
logreg_preds_train=logreg_model.fit(X_train, y_train).predict(X_train)
logreg_acc_train = balanced_accuracy_score(y_train, logreg_preds_train)
print('Logistic Regression Accuracy: {}'.format(logreg_acc))
print('Logistic Regression Training Accuracy: {}'.format(logreg_acc_train))
clinical_logreg=logreg_model.fit(X_train, y_train).predict(fst_cli)

In [ ]:
# evaluate an lasso regression model on the dataset
from numpy import arange
from pandas import read_csv
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso

# define model

model = Lasso()
# define model evaluation method
cv = RepeatedKFold(n_splits=3, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X_train, y_train)

## Tunning hyper parameters

In [ ]:
## Tunning parameters for RF
n_estimators=[2,3,4,5,6,7,8,9,10,11,12,13,14,15]
max_depth=[10,12,14,15,16,18,20]
min_samples_split=[10,20,30,40,50,60,70,80,90,100]
result_para=[]
result_perf=[]
for i1 in range(0,len(n_estimators)):
    print(i1)
    for i2 in range(0,len(max_depth)):
        for i3 in range(0,len(min_samples_split)):
            regc = RandomForestClassifier(n_estimators=n_estimators[i1],max_depth=max_depth[i2], min_samples_split=min_samples_split[i3],min_samples_leaf=1,random_state=0)

            regc.fit(X_train2, y_train2)
            y_pred_test=regc.predict(X_test2)
            y_pred_train=regc.predict(X_train2)

            y_pred_all_clinical=regc.predict(fst_cli3)
            y_pred_train_clinical=regc.predict(fst_cli3.iloc[df_clinical_train['num']])
            y_pred_test_clinical=regc.predict(fst_cli3.iloc[df_clinical_test['num']])


            rf_acc_test = balanced_accuracy_score(y_test2, y_pred_test)
            rf_acc_train = balanced_accuracy_score(y_train2, y_pred_train)

            rf_acc_all_clinical=balanced_accuracy_score((PSRv2>0.27)*1, y_pred_all_clinical)
            rf_acc_train_clinical=balanced_accuracy_score(df_clinical_train['label'], y_pred_train_clinical)
            rf_acc_test_clinical=balanced_accuracy_score(df_clinical_test['label'], y_pred_test_clinical)

            cli_rf_class_all=regc.predict(fst_cli3)
            cli_rf_class_train=regc.predict(fst_cli3.iloc[df_clinical_train['num']])
            cli_rf_class_test=regc.predict(fst_cli3.iloc[df_clinical_test['num']])

            logit_roc_auc3 = roc_auc_score(cli_rf_class_all,PSRv2)
            logit_roc_auc1 = roc_auc_score(cli_rf_class_train,PSRv2[df_clinical_train['num']])
            logit_roc_auc2 = roc_auc_score(cli_rf_class_test,PSRv2[df_clinical_test['num']])
            result_para.append([n_estimators[i1],max_depth[i2],min_samples_split[i3]])
            result_perf.append([rf_acc_test,rf_acc_train,rf_acc_all_clinical,logit_roc_auc3,logit_roc_auc1,logit_roc_auc2])
            
            

## feature importance

In [ ]:
# random forest for feature importance on a classification problem
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot
# define the model
model = RandomForestClassifier(n_estimators=5,max_depth=15, min_samples_split=30,min_samples_leaf=1,random_state=0)
# fit the model
model.fit(fst_BI1, Type_v2)
# get importance
importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

## RidgeClassifier

In [ ]:
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier().fit(X_train, y_train)
ridge_clf.score(X_train, y_train)
ridg_acc_train=balanced_accuracy_score(ridge_clf.predict(X_train), y_train)
ridg_acc_test=balanced_accuracy_score(ridge_clf.predict(X_test), y_test)
clinical_logreg=ridge_clf.predict(fst_cli)

## SVM

In [ ]:
from sklearn import svm
svm_linear_clf = svm.SVC(kernel='linear',C=1000)
svm_linear_clf.fit(X_train, y_train)
svm_linear_acc_train=balanced_accuracy_score(svm_linear_clf.predict(X_train), y_train)
svm_linear_acc_test=balanced_accuracy_score(svm_linear_clf.predict(X_test), y_test)
print(svm_linear_acc_train,svm_linear_acc_test)
clinical_svmlinear=svm_linear_clf.predict(fst_cli)